# Battle of the Neighborhoods Part 2

## The Problem
Detroit, Michigan went through one of the roughest economic recissions and housing crisises in the U.S. back in 2008. As of 2019 the city is rebounding faster than ever. Neighborhoods that were high crime now hold multi-million dollar condo developments thanks to reinvestment. Detroit is becoming the go to place for tech companies such as Google, Amazon, Microsoft, and IBM. While the market continues to grow so does finding a place to live and have entertainment(aka the 3rd place).

## The Data
Using Foursquare data we will explore the neighborhoods of Detroit, find an affordable place to call home, and an area to have fun in. I believe using this data we will be able to help visitors and new-comers in Detroit understand where to go.

### Preparing and cleansing

The data was gathered using the Foursqaure API and augmented with a csv from detroit.gov. We are focused on the area around 8 Mile which once had a movie written about it and is considered one of the main veins of the city. Below is a snippet of code used to gather the data.



In [1]:
import numpy as np 
import time
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 
import requests 
from pandas.io.json import json_normalize 


from geopy.geocoders import Nominatim 


import folium 
import folium 
from folium import plugins


import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns


from sklearn.cluster import KMeans



print('Libraries imported.')

Libraries imported.


# Setting Location After Libraries are Loaded

In [2]:
address = '8 Mile, Detroit'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of 8 Mile in Detroit are {}, {}.'.format(latitude, longitude))

C:\Users\turbo\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


The geograpical coordinates of 8 Mile in Detroit are 42.3315509, -83.0466403.


In [3]:
neighborhood_latitude=42.3315509
neighborhood_longitude=-83.0466403

In [17]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [18]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=&client_secret=&v=20180604&ll=42.3315509,-83.0466403&radius=500&limit=100'

In [9]:
results = requests.get(url).json()

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
      
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
      
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
      
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
venues = results['response']['groups'][0]['items']
SGnearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]
# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)
# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.shape

(100, 4)

# Data output

We can see that in the 8 Mile area it is mostly restaurants. If we were to move here there is an abundnace of choice to live by!

In [16]:
SGnearby_venues.head(50)

,name,categories,lat,lng
0,Avalon Cafe and Bakery,Café,42.332834,-83.047694
1,Campus Martius,Park,42.331575,-83.046598
2,Texas de Brazil,Steakhouse,42.332293,-83.046711
3,Dime Store,American Restaurant,42.331039,-83.047734
4,Chase Bank,Bank,42.330676,-83.046648
5,Bon Bon Bon,Dessert Shop,42.330548,-83.047914
6,Detroit Water Ice Factory,Ice Cream Shop,42.332575,-83.047436
7,Lafayette Coney Island,Hot Dog Joint,42.331683,-83.048780
8,Grand Trunk Pub,Pub,42.330403,-83.045943
9,Guardian Building,Building,42.329497,-83.046083


# Rental data


Month	All 1B	2B
1/2011	724	624	802
2/2011	745	655	831
3/2011	760	661	856
4/2011	824	726	885
5/2011	840	687	928
6/2011	784	651	824
7/2011	585	514	628
8/2011	681	588	726
9/2011	753	640	855
10/2011	786	660	919
11/2011	744	646	875
12/2011	664	590	744
1/2012	715	625	800
2/2012	758	620	811
3/2012	755	564	711
4/2012	776	584	735
5/2012	877	609	926
6/2012	839	612	848
7/2012	805	630	803
8/2012	814	633	895
9/2012	830	623	964
10/2012	875	645	997
11/2012	850	644	983
12/2012	778	604	895
1/2013	747	606	851
2/2013	739	603	842
3/2013	773	629	903
4/2013	779	622	908
5/2013	763	614	887
6/2013	783	647	895
7/2013	775	635	886
8/2013	746	629	820
9/2013	735	621	841
10/2013	740	634	837
11/2013	784	613	933
12/2013	814	684	968
1/2014	823	731	957
2/2014	763	635	878
3/2014	781	666	883
4/2014	803	654	941
5/2014	777	653	913
6/2014	802	696	932
7/2014	848	779	977
8/2014	808	738	934
9/2014	784	718	889
10/2014	761	691	857
11/2014	741	636	866
12/2014	807	719	938
1/2015	803	683	966
2/2015	799	688	957
3/2015	851	769	1,026
4/2015	851	766	998
5/2015	773	671	902
6/2015	743	628	823
7/2015	768	654	863
8/2015	762	637	860
9/2015	746	628	847
10/2015	752	646	849
11/2015	762	661	859
12/2015	786	680	890
1/2016	821	722	918
2/2016	843	722	943
3/2016	950	847	1,052
4/2016	961	851	1,081
5/2016	986	877	1,111
6/2016	956	839	1,100
7/2016	958	837	1,096
8/2016	992	877	1,132
9/2016	1,046	920	1,198
10/2016	1,152	992	1,276
11/2016	1,133	968	1,288
12/2016	1,142	944	1,291
1/2017	1,097	888	1,289
2/2017	1,061	852	1,251
3/2017	1,158	949	1,274
4/2017	1,175	930	1,302
5/2017	1,191	969	1,254
6/2017	1,191	992	1,235
7/2017	1,255	1,056	1,283
8/2017	1,284	1,060	1,331
9/2017	1,236	1,029	1,315
10/2017	1,139	975	1,254
11/2017	1,199	1,003	1,287
12/2017	1,171	979	1,284
1/2018	1,171	980	1,243
2/2018	1,155	965	1,239
3/2018	1,156	984	1,261
4/2018	1,155	975	1,279
5/2018	1,013	814	1,204
6/2018	1,090	861	1,292
7/2018	1,159	911	1,401
8/2018	1,166	902	1,425
9/2018	1,110	823	1,353
10/2018	1,121	846	1,411
11/2018	1,094	819	1,358
12/2018	1,012	786	1,233
1/2019	1,024	820	1,229
2/2019	1,054	837	1,261
3/2019	1,109	891	1,312
4/2019	1,176	941	1,401
10/2019	1,391	1,001	1,345

We can see that from 1/2011 - 10/2019 that the rent has increased by almost 2x as much. At the current rate in 5 years a rental near 8 Mile on average would cost around $2600 per month!